In [1]:
#Подключение основных библиотек
from datetime import datetime

In [2]:
import pandas

In [3]:
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
from sklearn.metrics import roc_auc_score

In [5]:
from sklearn.model_selection import cross_val_score

In [6]:
from sklearn.model_selection import KFold

In [7]:
#Получение обучающей выборки
data_train = pandas.read_csv('features.csv', index_col='match_id')

In [8]:
#Извлекаем "правильные ответы" из выборки
y = data_train['radiant_win']

In [9]:
#Удаление целевой переменной и признаков "заглядывающих в будущее", которые мешают строить адекватную модель для предсказаний
X = data_train.drop(['radiant_win', 'duration', 'tower_status_radiant','tower_status_dire','barracks_status_radiant', 'barracks_status_dire'], axis=1)

In [10]:
#Заполняем пропущенные значения средними по своему признаку
X = X.fillna(X.mean())

In [11]:
# Обучение градиетного бустинга
# Входные данные - число деревьев, выходные - roc_auc метрика и время выполнения скрипта
def teaching_grb(number_of_trees):
    #Засекаем время начала работы алгоритма
    startTime = datetime.now()
    kf = KFold(n_splits=5, shuffle=True, random_state=241)
    grad = GradientBoostingClassifier(n_estimators=number_of_trees)
    grad.fit(X, y)
    return cross_val_score(grad, X, y, scoring='roc_auc', cv=kf,n_jobs=-1).mean(), datetime.now() - startTime

In [12]:
#Обучение градиентного бустинга на 10, 30 и 100 деревьях
for n in [10, 30, 100]:
    roc_auc, execution_time = teaching_grb(n)
    print('Для ' + str(n) + ' деревьев метрика roc_auc=' + str(round(roc_auc, 2)) + ' | Время выполнения:' + str(execution_time))

Для 10 деревьев метрика roc_auc=0.66 | Время выполнения:0:00:21.257054
Для 30 деревьев метрика roc_auc=0.69 | Время выполнения:0:00:58.156045
Для 100 деревьев метрика roc_auc=0.71 | Время выполнения:0:02:50.526739


In [13]:
#Подключение библиотек для работы с логической регрессией
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [14]:
scaler = StandardScaler()

In [15]:
#Для поиска максимального значения, его C коэффициент регуляризации и на каком этапе 
roc_auc_best = 0.0, 0.0, ''

In [16]:
# Обучение линейной регрессии
# Входные данные - обучающая выборка, выходные - roc_auc метрика и время выполнения
def teaching_lr(X_train, C_val):
    startTime = datetime.now()
    LogReg = LogisticRegression(C=C_val)
    LogReg.fit(X_train, y)
    kf = KFold(n_splits=5, shuffle=True)
    res = cross_val_score(LogReg, X_train, y, scoring='roc_auc', cv=kf,n_jobs=-1).mean()
    return res, datetime.now() - startTime

In [17]:
C_list = [0.1, 0.2, 1.0, 2.0, 10.0, 20, 100.0, 200.0]

In [18]:
#Проверим качество метрики с исходными данными и сравним время выполнения
print('Логическая регрессия:')
print('Для исходных данных:')

Логическая регрессия:
Для исходных данных:


In [19]:
for C in C_list:
    roc_auc, execution_time = teaching_lr(scaler.fit_transform(X), C)
    if roc_auc_best[0] < roc_auc:
        roc_auc_best = roc_auc, C, 'Исходные данные'
    print('Для c=' + str(C) + ' roc_auc=' + str(round(roc_auc, 5)) + ' | Время выполнения:' + str(execution_time))

Для c=0.1 roc_auc=0.71668 | Время выполнения:0:00:12.329558
Для c=0.2 roc_auc=0.71682 | Время выполнения:0:00:13.696437
Для c=1.0 roc_auc=0.71674 | Время выполнения:0:00:12.221884
Для c=2.0 roc_auc=0.71681 | Время выполнения:0:00:12.666733
Для c=10.0 roc_auc=0.71676 | Время выполнения:0:00:12.409124
Для c=20 roc_auc=0.71674 | Время выполнения:0:00:12.251275
Для c=100.0 roc_auc=0.71677 | Время выполнения:0:00:11.948999
Для c=200.0 roc_auc=0.71703 | Время выполнения:0:00:11.888332


In [20]:
#Очистим выборку от категориальных признаков, здесь их 11, а именно тип лобби и герои
X_cleared = X.drop(['lobby_type'], axis=1)

In [21]:
for i in range(1,6):
    X_cleared = X_cleared.drop(['r' + str(i) + '_hero'], axis=1)
    X_cleared = X_cleared.drop(['d' + str(i) + '_hero'], axis=1)

In [22]:
#Проверим качество метрики после удаления категориальных признаков
print('roc_auc после удаления категориальных признаков')
for C in C_list:
    roc_auc_cleared, execution_time = teaching_lr(scaler.fit_transform(X_cleared), C)
    if roc_auc_best[0] < roc_auc_cleared:
        roc_auc_best = roc_auc_cleared, C, 'После удаления признаков'
    print('Для c=' + str(C) + ' roc_auc=' + str(round(roc_auc_cleared, 5)) + ' | Время выполнения:' + str(execution_time))

roc_auc после удаления категориальных признаков
Для c=0.1 roc_auc=0.71687 | Время выполнения:0:00:12.139404
Для c=0.2 roc_auc=0.71681 | Время выполнения:0:00:13.198120
Для c=1.0 roc_auc=0.71678 | Время выполнения:0:00:11.931789
Для c=2.0 roc_auc=0.71685 | Время выполнения:0:00:11.400787
Для c=10.0 roc_auc=0.71678 | Время выполнения:0:00:11.079715
Для c=20 roc_auc=0.71676 | Время выполнения:0:00:10.815804
Для c=100.0 roc_auc=0.71707 | Время выполнения:0:00:10.307537
Для c=200.0 roc_auc=0.71687 | Время выполнения:0:00:10.333745


In [23]:
#Поиск максимального идентификатора героя в тестовой выборки для всех 10 игроков

In [24]:
max_id = 0

In [25]:
for n in range(1,6):
    if data_train['r' + str(n) + '_hero'].max() > max_id:
        max_id = data_train['r' + str(n) + '_hero'].max()
    elif data_train['d' + str(n) + '_hero'].max() > max_id:
        max_id = data_train['d' + str(n) + '_hero'].max()

In [26]:
print(max_id)

112


In [27]:
#Проверим это число с количеством различных идентификаторов героев во всех играх любым из игроков

In [28]:
isset = list()

In [29]:
for n in range(1,6):
    r = data_train['r' + str(n) + '_hero'].unique()
    d = data_train['d' + str(n) + '_hero'].unique()
    isset = list(set().union(isset, r, d))

In [30]:
#Получаем число всех выбираемых героев
uniqie_ids = len(isset)

In [31]:
#То есть в выборке не присутствовали 4 идентификатора
print(max_id - uniqie_ids)

4


In [32]:
#Получим список "неигральбельных" героев
for i in range(1, max_id + 1):
    if not i in isset:
        print(i)

24
107
108
111


In [33]:
import numpy

In [34]:
#Получение "мешка слов" по героям
X_pick = numpy.zeros((X.shape[0], max_id))

for i, match_id in enumerate(X.index):
    for p in range(1, 5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

d:\programming\python\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [35]:
Bag_of_words = pandas.DataFrame(X_pick, index=X.index)

In [36]:
#Объединение признаков с "мешком слов"
X_with_bag = pandas.concat([X_cleared, Bag_of_words], axis=1)

In [37]:
#Проверим качество метрики после добавление "мешка слов" по героям
for C in C_list:
    roc_auc_bag, execution_time = teaching_lr(scaler.fit_transform(X_with_bag), C)
    if roc_auc_best[0] < roc_auc_bag:
        roc_auc_best = roc_auc_bag, C, 'Мешок слов'
    print('Для c=' + str(C) + ' roc_auc=' + str(round(roc_auc_bag, 5)) + ' | Время выполнения:' + str(execution_time))

Для c=0.1 roc_auc=0.74322 | Время выполнения:0:00:20.578156
Для c=0.2 roc_auc=0.74336 | Время выполнения:0:00:22.030139
Для c=1.0 roc_auc=0.74358 | Время выполнения:0:00:20.520285
Для c=2.0 roc_auc=0.74347 | Время выполнения:0:00:20.823919
Для c=10.0 roc_auc=0.74339 | Время выполнения:0:00:21.146989
Для c=20 roc_auc=0.74338 | Время выполнения:0:00:22.444788
Для c=100.0 roc_auc=0.74335 | Время выполнения:0:00:22.206341
Для c=200.0 roc_auc=0.74324 | Время выполнения:0:00:20.793670


# Ответы на вопросы

# Подход 1: градиентный бустинг "в лоб"

In [38]:
nan_columns = data_train.isnull().sum().sort_values(ascending=False)

In [39]:
print('Признаки, имеющие пропуски значений')
for c in nan_columns.keys():
    if nan_columns[c] <= 0:
        break
    print('Признак:' + str(c) + '              Кол-во пропусков: ' + str(nan_columns[c]))

Признаки, имеющие пропуски значений
Признак:first_blood_player2              Кол-во пропусков: 43987
Признак:radiant_flying_courier_time              Кол-во пропусков: 27479
Признак:dire_flying_courier_time              Кол-во пропусков: 26098
Признак:first_blood_player1              Кол-во пропусков: 19553
Признак:first_blood_team              Кол-во пропусков: 19553
Признак:first_blood_time              Кол-во пропусков: 19553
Признак:dire_bottle_time              Кол-во пропусков: 16143
Признак:radiant_bottle_time              Кол-во пропусков: 15691
Признак:radiant_first_ward_time              Кол-во пропусков: 1836
Признак:dire_first_ward_time              Кол-во пропусков: 1826
Признак:radiant_courier_time              Кол-во пропусков: 692
Признак:dire_courier_time              Кол-во пропусков: 676


Если рассмотреть два логически связанных признака "first_blood_team"(команда, совершившая первую кровь) и "first_blood_time"(игровое время первой крови), то видно что они имеют одинаковое количество. Во-первых, это говорит о том, что они выполняются вместе. Во-вторых, незаполненность данных скорее всего связана с тем, что данные события происходят после 5 минут от начала игры и не попадали в тестовоую выборку.

radiant_win

Длилось: 51 секунду. <br />
Качество: 0.69
P.S. без кросс-валидации качество метрики получилось примерно такое же, но время выполнения ~13 секунд, что в 4 раза быстрее

Спорный вопрос. Например, при 30 деревьях roc_auc=0.69, а при 100 деревьях roc_auc=0.71 <br />
Качество улучшилось, но время алгоритма выросло в несколько раз. Прирост в качестве не такой большой по сравнению с потраченным временем. Также значительно уступает логической регрессии по качеству и времени выполнения. <br />
Для ускорения можно, например, изменить глубину поиска(max_depth) или n_jobs=-1 для кросс-валидации, что даст небольшое сокращение времени, но значительно нагрузит все процессоры вплоть до 99.9%  

# Подход 2: логистическая регрессия

При исходных данных roc_auc = 0.72 <br />
Если сравнивать логичестическую регрессию с исходными данными(только после их масштабирования), то она уже выигравывает в качестве градиентный бустинг при 30 деревьях. <br />
Разница в качестве вызвана тем, что градиентный бустинг "в лоб" пытается найти решение.
Логистическая регрессия значительно превосходит по скорости бустинг. Например, при 100 деревьях подсчет занимает 3 минуты(без n_jobs=-1), в то время как регрессия тратит на это несколько секунд.

Незначительная разница после удаления. Это может означать, что эти признаки имели крайне малый вес при подсчете предсказания.

Если рассматривать только данные из тестовой выборки, то число различных идентификаторов равно: 108, но максимальный - 112

Качество улучшилось. С 0.72 с исходными данными до 0.74. Это может быть вызвано тем, что вместо случайных категориальных иденфикаторов, мы получили осмысленную связь между определенным героев(группы героев) и целевой переменной

In [40]:
print('Лучший алгоритм:')
print('roc_auc:' +str(round(roc_auc_best[0], 2)) + '\nC:' + str(roc_auc_best[1]) + '\nЭтап:' + str(roc_auc_best[2]))

Лучший алгоритм:
roc_auc:0.74
C:1.0
Этап:Мешок слов


In [41]:
LogReg = LogisticRegression(C=roc_auc_best[1])

In [42]:
LogReg.fit(scaler.fit_transform(X_with_bag), y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [43]:
pred = LogReg.predict_proba(scaler.fit_transform(X_with_bag))[:, 1]

In [45]:
print('min:'+str(round(numpy.min(pred), 3)) + ' max:' +str(round(numpy.max(pred), 3)))

min:0.001 max:0.999
